In [223]:
import numpy as np
import random
from PIL import Image
from collections import Counter

In [224]:
def hamming_encode(data):
    # Encode data bits using Hamming (7,4) encoding.

    p1 = data[0] ^ data[1] ^ data[3]  # Parity bit for positions 1, 2, 4
    p2 = data[0] ^ data[2] ^ data[3]  # Parity bit for positions 1, 3, 4
    p3 = data[1] ^ data[2] ^ data[3]  # Parity bit for positions 2, 3, 4

    # Return the 7-bit Hamming encoded data
    return np.array([p1, p2, data[0], p3, data[1], data[2], data[3]])

In [225]:
# Get the dimensions of the image
img_name = input("Enter Image Name with extension: ")
original_image_array = np.array(Image.open(img_name))
height, width, channels = original_image_array.shape

In [226]:
# Prepare output arrays for encoded channels
encoded_red = np.zeros((height, width, 3), dtype=np.uint8)
encoded_green = np.zeros((height, width, 3), dtype=np.uint8)
encoded_blue = np.zeros((height, width, 3), dtype=np.uint8)


In [227]:
for i in range(height):
    for j in range(width):
        for n in range(3):
           # Get the pixel value for the current color channel
            pixel_value = original_image_array[i, j, n]

            # Convert pixel value to an 8-bit binary string
            binary_string = format(pixel_value, '08b')  # '08b' ensures it is padded to 8 bits

            # Split the binary string into two halves
            upper_half = binary_string[:4]  # First four bits
            lower_half = binary_string[4:]  # Last four bits

            

            upper_half_array = []
            lower_half_array = []


            for m in range(len(upper_half)):
                upper_half_array.append(int(upper_half[m]))

            for k in range(len(lower_half)):
                lower_half_array.append(int(lower_half[k]))


            # Encode each half
            encoded_upper = hamming_encode(upper_half_array)
            encoded_lower = hamming_encode(lower_half_array)

            # Concatenate the encoded bits (upper + lower)
            encoded_combined = np.concatenate((encoded_upper, encoded_lower))

            # Append 10 trailing zeros to get 24 bits for one pixel in each extracted colour image
            encoded_combined_with_zeros = np.concatenate((encoded_combined, [0] * 10))

            

            # Store the first 8 bits in the pseudo red channel 
            pseudo_red_value = int("".join(map(str, encoded_combined_with_zeros[:8])), 2) 

            # Store the second 8 bits in the pseudo  green channel 
            pseudo_green_value = int("".join(map(str,encoded_combined_with_zeros[8:16])), 2) 
            
            # Store the third 8 bits in the pseudo  blue channel 
            pseudo_blue_value = int("".join(map(str,encoded_combined_with_zeros[16:24])), 2)


            # Store the values in the respective channels of the encoded image encoded_image[i, j] = (

            # Determine the colour channel to extract red, blue, green data separately

            if n == 0:
                encoded_red[i, j, 0] = pseudo_red_value
                encoded_red[i, j, 1] = pseudo_green_value
                encoded_red[i, j, 2] = pseudo_blue_value

            elif n == 1:
                encoded_green[i, j, 0] = pseudo_red_value
                encoded_green[i, j, 1] = pseudo_green_value
                encoded_green[i, j, 2] = pseudo_blue_value
            else:
                encoded_blue[i, j, 0] = pseudo_red_value
                encoded_blue[i, j, 1] = pseudo_green_value
                encoded_blue[i, j, 2] = pseudo_blue_value




In [228]:
# Save the encoded images
encoded_red_img = Image.fromarray(encoded_red)
encoded_green_img = Image.fromarray(encoded_green)
encoded_blue_img = Image.fromarray(encoded_blue)

encoded_red_img.save("encoded_red.png")
encoded_green_img.save("encoded_green.png")
encoded_blue_img.save("encoded_blue.png")



In [229]:
def introduce_noise(image, seed=None):
    if seed is not None:
        np.random.seed(seed)  # Set the seed for reproducibility
        random.seed(seed)      # Set the seed for the random module

    noisy_image = np.copy(image)
    height, width = noisy_image.shape
    
    # Iterate over each pixel
    for i in range(height):
        for j in range(width):
            # Get the current pixel value
            pixel_value = noisy_image[i, j]
            binary_pixel = list(format(pixel_value, '08b'))  # Convert pixel to binary list
            
            # Introduce an error in one random bit in the 8-bit pixel
            error_bit = random.randint(0, 7)
            binary_pixel[error_bit] = '1' if binary_pixel[error_bit] == '0' else '0'  # Flip the bit
            
            # Convert the binary list back to an integer
            noisy_pixel = int(''.join(binary_pixel), 2)
            noisy_image[i, j] = noisy_pixel
    
    return noisy_image

In [230]:
# Function to process and save noisy images for given file path
def process_and_save_noisy_images(file_path, output_file):
    # Load the image
    original_img = Image.open(file_path)
    original_array = np.array(original_img)
    
    # Assuming the image is grayscale or has been preprocessed to be treated as such
    if len(original_array.shape) == 3:
        height, width, channels = original_array.shape
        # Introduce noise separately for each channel if it's a color image
        noisy_channels = [introduce_noise(original_array[:, :, i]) for i in range(channels)]
        noisy_image_array = np.stack(noisy_channels, axis=-1)
    else:
        noisy_image_array = introduce_noise(original_array)
    
    # Convert array back to an image
    noisy_image = Image.fromarray(noisy_image_array.astype('uint8'))
    noisy_image.save(output_file)


process_and_save_noisy_images('encoded_red.png', 'noisy_encoded_red.png')
process_and_save_noisy_images('encoded_green.png', 'noisy_encoded_green.png')
process_and_save_noisy_images('encoded_blue.png', 'noisy_encoded_blue.png')

In [231]:
def hamming_decode(data):
    # Decode data bits using Hamming (7,4) encoding.
    p1 = data[3] ^ data[4] ^ data[5] ^ data[6]  # Parity bit for positions 3, 4, 5, 6
    p2 = data[1] ^ data[2] ^ data[5] ^ data[6]  # Parity bit for positions 1, 2, 5, 6
    p3 = data[0] ^ data[2] ^ data[4] ^ data[6]  # Parity bit for positions 0, 2, 4, 6

    string = f"{p1}{p2}{p3}"
    
    if (int(string,2) != 0):
        data[int(string,2)-1] ^= 1

    return np.array([data[2], data[4], data[5], data[6]]) # Load the noisy images

In [232]:
# Load the noisy images
noisy_red = np.array(Image.open("noisy_encoded_red.png"))
noisy_green = np.array(Image.open("noisy_encoded_green.png"))
noisy_blue = np.array(Image.open("noisy_encoded_blue.png"))

# Extract height and width
height, width, _ = noisy_red.shape

# Prepare an output array for the decoded image
decoded_image = np.zeros((height, width, 3), dtype=np.uint8)

In [233]:
# Process each pixel with a triple for loop
for i in range(height):
    for j in range(width):
        for n in range(3):
            # Select the appropriate noisy image
            if n == 0:
                noisy_image = noisy_red
                color_index = 0
            elif n == 1:
                noisy_image = noisy_green
                color_index = 1
            elif n == 2:
                noisy_image = noisy_blue
                color_index = 2
            
            # Extract pixel data and convert it to bits
            pixel_value = noisy_image[i, j]

            bitstring = ""

            for k in range(len(pixel_value)):
                bitstring += str(format(pixel_value[k],'08b')) 

            upper_bits = bitstring[0:7]
            lower_bits = bitstring[7:14]

            upper_bits_array = []
            lower_bits_array = []

            for l in range(len(upper_bits)):
                upper_bits_array.append(int(upper_bits[l]))

            for m in range(len(lower_bits)):
                lower_bits_array.append(int(lower_bits[m]))

            # Decode each half
            decoded_upper = hamming_decode(upper_bits_array)
            decoded_lower = hamming_decode(lower_bits_array)

            # Concatenate the de bits (upper + lower)
            decoded_combined = np.concatenate((decoded_upper, decoded_lower))


            corrected_value = int("".join(map(str, decoded_combined)), 2) 

             # Store the 8-bit value in the decoded image array
            decoded_image[i, j, color_index] = corrected_value




In [234]:
# Save the decoded image
decoded_img = Image.fromarray(decoded_image)
decoded_img.save("decoded_image.png")